In [11]:
import sys
import numpy as np
import lightgbm as lg
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import catboost
from catboost.utils import get_gpu_device_count

sys.path.append("../../scorer/")
# import orderbook as ob
# Чтобы использовать быстрый ордербук раскомментируйте строку:
import orderbook_fast as ob
 
SIDE_BID = 0 
SIDE_ASK = 1

## Собираем датасет для тренировки модели

In [2]:
def get_simple_features_from_orderbook(orderbook, max_index=2):
    '''
        Считаем простые фичи по ордербуку:
    '''
    spread = orderbook.get_best_price(SIDE_ASK) - orderbook.get_best_price(SIDE_BID)
    features = [spread]
    for side in (SIDE_BID, SIDE_ASK):
        for ix in range(max_index):
            price_level = orderbook.get_price_level_at_ix(side, ix)
            if price_level is None:
                features += [-1, -1]
            else:
                features += [price_level.get_volume(), 
                             price_level.get_num_orders()]
    return features


def get_simple_deals_features(last_deals, orderbook):
    '''
        Считаем простые фичи по последним сделкам:
    '''
    cur_mean_price = orderbook.get_mean_price()
    cur_time = orderbook.get_time()

    features = []
    for side in (SIDE_BID, SIDE_ASK):
        deal_event = last_deals[side]
        if deal_event is None:
            features += [-1e9, -1e9, -1e9]
        else:
            features += [cur_mean_price - deal_event.price, 
                         cur_time - deal_event.time, 
                         deal_event.amount]
    return features


def collect_dataset(data_path):
    '''
        Собираем датасет
    '''
    event_player = ob.EventPlayer(data_path)
    orderbook = ob.OrderBook()

    X = []
    Y = []

    last_deals = [None, None]
    for ev in tqdm(event_player.iget_events(), 
                    total=len(event_player), 
                    desc="collecting dataset"):
        if ev.action == ob.Action.DEAL:
            last_deals[ev.side] = ev
        elif ev.action == ob.Action.NEW_CHUNK:
            last_deals = [None, None]
        
        orderbook.apply_event(ev)
        if ev.need_prediction:
            features = get_simple_features_from_orderbook(orderbook)
            features += get_simple_deals_features(last_deals, orderbook)

            X.append(features)
            Y.append(ev.Y)

    print(f"Dataset collected: len(X) = {len(X)}")
    return np.array(X), np.array(Y)


X_train, Y_train = collect_dataset("../../data/train_small_A.npz")
X_test, Y_test = collect_dataset("../../data/train_small_B.npz")

collecting dataset: 100%|██████████| 10555835/10555835 [01:06<00:00, 157680.82it/s]


Dataset collected: len(X) = 234905
Dataset collected: len(X) = 240309


## Обучаем модель градиентного бустинга

In [8]:
def train_classifier(X_train, Y_train, X_test, Y_test):
    '''
        Обучаем классификатор
    '''
    clf = lg.LGBMClassifier(num_leaves=31, n_estimators=1000, learning_rate=0.1)
    clf.fit(X_train, Y_train, eval_set=[(X_test, Y_test)], 
            eval_metric="auc", early_stopping_rounds=20)
    return clf
 
# clf = train_classifier(X_train, Y_train, X_test, Y_test)

In [9]:
%%time
def train_catboost(X_train, Y_train, X_test, Y_test):

    task_type = "CPU" #  if (get_gpu_device_count() == 0) else "GPU"
    langevin = False if task_type == "GPU" else True
    print(f"catboost training with {task_type}...")
    print("X_train.shape = ", X_train.shape)
    print("get_gpu_device_count() = ", get_gpu_device_count())

    train_pool = catboost.Pool(X_train, Y_train)

    grid = {'learning_rate': [0.03, 0.01],
            # 'score_function': ["Cosine", "L2", "NewtonL2"],
            'depth': [4, 5, 6],
            'l2_leaf_reg': [2, 3, 5, 8, 11],
            # 'nan_mode': ["Min", "Max"],
            # 'fold_len_multiplier': np.linspace(1.1, 3, 3),
            # 'bagging_temperature': np.linspace(1, 3000, 3),

    }

    model = catboost.CatBoostClassifier(

        bootstrap_type="Bayesian",
        # scale_pos_weight=en.scale_pos_weight,
        # scale_pos_weight=1,
        loss_function = "Logloss",
        eval_metric="AUC:hints=skip_train~false",
        langevin=langevin,
        custom_metric=["Recall", "Precision", "Accuracy", "F1", "Kappa", "MCC"],
        # use_best_model=True,
        iterations=10,
        od_type="Iter",
        od_wait=30,
        # rsm=0.5,
        # random_seed=100,
        task_type=task_type,
        boosting_type='Ordered',
        logging_level="Verbose",
        train_dir="grid",

    )

    grid_search_result = model.grid_search(
        grid,
        X=train_pool,
        stratified=True,
        cv=3,
        search_by_train_test_split=False,
        plot=False      # PLOT,

    )


    params = model.get_params()
    for param in params.items():
        print(param)

    return grid_search_result, model

grid_search_result, clf = train_catboost(X_train, Y_train, X_test, Y_test)
# clf = train_classifier(X_train, Y_train, X_test, Y_test)

catboost training with CPU...
X_train.shape =  (234905, 15)
get_gpu_device_count() =  1
0:	loss: 0.6060353	best: 0.6060353 (0)	total: 12.1s	remaining: 5m 49s
1:	loss: 0.6060113	best: 0.6060353 (0)	total: 23.2s	remaining: 5m 25s
2:	loss: 0.6059978	best: 0.6060353 (0)	total: 35.6s	remaining: 5m 20s
3:	loss: 0.6059995	best: 0.6060353 (0)	total: 47.2s	remaining: 5m 6s
4:	loss: 0.6059811	best: 0.6060353 (0)	total: 59.1s	remaining: 4m 55s
5:	loss: 0.6059804	best: 0.6060353 (0)	total: 1m 11s	remaining: 4m 45s
6:	loss: 0.6059590	best: 0.6060353 (0)	total: 1m 24s	remaining: 4m 36s
7:	loss: 0.6059391	best: 0.6060353 (0)	total: 1m 37s	remaining: 4m 27s
8:	loss: 0.6057664	best: 0.6060353 (0)	total: 1m 49s	remaining: 4m 16s
9:	loss: 0.6058218	best: 0.6060353 (0)	total: 2m 2s	remaining: 4m 5s
10:	loss: 0.6076494	best: 0.6076494 (10)	total: 2m 15s	remaining: 3m 54s
11:	loss: 0.6066110	best: 0.6076494 (10)	total: 2m 28s	remaining: 3m 42s
12:	loss: 0.6076393	best: 0.6076494 (10)	total: 2m 40s	remaining

## Тестируем получившийся классификатор

In [5]:
last_deals = [None, None]

def process_event_and_predict_proba(event, orderbook):
    if event.action == ob.Action.DEAL:
        last_deals[event.side] = event
    elif event.action == ob.Action.NEW_CHUNK:
        last_deals[:] = [None, None]
        
    if not event.need_prediction:
        return None
    
    features = get_simple_features_from_orderbook(orderbook)
    features += get_simple_deals_features(last_deals, orderbook)    
    proba = clf.predict_proba([features])[0, 1]
    return proba

from scorer import Scorer

scoring = Scorer("../../data/train_small_C.npz")
# roc_auc, (true_ys, pred_probas) = scoring.score(process_event_and_predict_proba)

## Сохраним модель, и решение для отправки готово

In [13]:
# Сохраним нашу модель
# clf.booster_.save_model("wunder.model")
clf.save_model("wunder.model", format="cbm")
print(f"curdir: {Path.cwd()}")
'''Посмотрите код файла solution.py. 
Он использует те же функции что и этот ноутбук, но уже готов к отправке на серверю. 
Попробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'''

curdir: /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/wunder_summer/wunder_challenge/examples/catboost


'Посмотрите код файла solution.py. \nОн использует те же функции что и этот ноутбук, но уже готов к отправке на серверю. \nПопробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'